# quiz-qe2107

Use the "Run" button to execute the code.

In [1]:
!pip install jovian --upgrade --quiet

In [2]:
import jovian

In [ ]:
# Execute this to save new versions of the notebook
jovian.commit(project="quiz-qe2107")

<IPython.core.display.Javascript object>

In [ ]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
from pandas import DataFrame
import pathlib
import matplotlib.pyplot as plt
from statsmodels.tsa.api import SimpleExpSmoothing,Holt,ExponentialSmoothing
plt.style.use('seaborn')

In [ ]:
import warnings
from statsmodels.tools.sm_exceptions import ConvergenceWarning
warnings.simplefilter('ignore', ConvergenceWarning)

In [ ]:
pip show statsmodels

In [ ]:
y_t = []
time = np.arange(500)   
y_t.append (2 + np.random.normal())
for i in range(499):     
    y_t.append(2 + (0.6)*i + (0.7)*y_t[i] + np.random.normal())

plt.plot(time,y_t)


In [ ]:
y_t = pd.Series(y_t)
y_t.index = pd.date_range(start = "30-01-2022", periods = 500, freq ='D')
plt.plot(y_t)

**Simple Exponential Smoothing**

In [ ]:
alpha = np.linspace(0.1,1,10,True)
SESfit =[]
plt.plot(y_t[490:])
for a in alpha:
  SES = SimpleExpSmoothing(y_t,initialization_method = 'estimated').fit(smoothing_level=a, optimized = False,use_brute= False)
  SESfit.append(SES.fittedvalues)
  SESforecast = SES.forecast(10)
  plt.plot(SESforecast,label=" estimated alpha = {:.2f}".format(a))
plt.legend()

plt.figure(figsize=(15,3))

In [ ]:
SESpmse=[np.mean((y_t-SESfit[i])**2) for i  in range(10)]
SESbest = np.where(SESpmse == np.min(SESpmse))                 
print("minimum SESpmse among all cases:=",np.min(SESpmse))
print("The optimal value of alpha is=",alpha[SESbest[0]])

In [ ]:
plt.plot(y_t[490:])
SES_bestmodel = SimpleExpSmoothing(y_t,initialization_method='estimated').fit(smoothing_level=alpha[SESbest[0]],optimized = False)
plt.plot(SES_bestmodel.forecast(10),label = "alpha={:.1f}".format(SES_bestmodel.params["smoothing_level"]))
plt.legend()
plt.show()

**Holts Exponential Smoothing**

In [ ]:
beta = np.linspace(0.1,1,10)
HESfit=[]
for a in alpha:
    HESarr=[]
    for b in beta:
        HES = Holt(y_t,initialization_method='heuristic').fit(smoothing_level=a, smoothing_trend=b, optimized = False,use_brute= False)
        HESarr.append(HES.fittedvalues)
    HESfit.append(HESarr)
HESPMSE = [[np.mean((y_t-HESfit[i][j])**2) for j in range(10)] for i in range(10)]

In [ ]:
print("minimum HESPMSE from parameters  best found by grid search method :",np.min(HESPMSE))

In [ ]:
HESbest = np.where(HESPMSE == np.min(HESPMSE))

In [ ]:
HES_bestmodel = Holt(y_t,initialization_method='estimated').fit(smoothing_level=alpha[HESbest[0][0]], smoothing_trend=beta[HESbest[1][0]], optimized = False)
HES_optimizedmodel = Holt(y_t,initialization_method='estimated').fit()

In [ ]:
print(" Optimal parameters best found by python function: Alpha={:.2f}, Beta={:.2f}".format(HES_optimizedmodel.params['smoothing_level'],HES_optimizedmodel.params['smoothing_trend']))
print(" Optimal parameters best found by grid search method: Alpha={:.1f}, Beta={:.1f}".format(alpha[HESbest[0][0]],beta[HESbest[1][0]]))

In [ ]:
plt.plot(y_t[490:])
plt.plot(HES_bestmodel.forecast(10),label = "found by grid search method:alpha={:.1f}, beta={:.1f}".format(HES_bestmodel.params["smoothing_level"],HES_bestmodel.params["smoothing_trend"]))
plt.plot(HES_optimizedmodel.forecast(10),label = "found by python function : alpha={:.2f}, beta={:.2f}".format(HES_optimizedmodel.params["smoothing_level"],HES_optimizedmodel.params["smoothing_trend"]))
plt.legend()
plt.show()

**Damped Holt Exp Smoothing**

In [ ]:
gamma = np.linspace(0.1,1,10)
DHESfit=[]
for a in alpha:
    DHESarr=[]
    for b in beta:
        DHES = Holt(y_t,initialization_method='estimated',damped_trend=True).fit(smoothing_level=a, smoothing_trend=b)
        DHESarr.append(DHES.fittedvalues)
    DHESfit.append(DHESarr)
DHESPMSE = [[np.mean((y_t-DHESfit[i][j])**2) for j in range(10)] for i in range(10)]
print("minimum DHESPMSE from given parameters:",np.min(DHESPMSE))
DHESbest = np.where(DHESPMSE == np.min(DHESPMSE))
DHES_bestmodel = Holt(y_t,initialization_method='estimated').fit(smoothing_level=alpha[DHESbest[0][0]], smoothing_trend=beta[DHESbest[1][0]])
DHES_optimizedmodel = Holt(y_t,initialization_method='estimated').fit()
print("Optimal parameters best found by python function: Alpha={:.2f}, Beta={:.2f},Gamma={:.2f}".format(DHES_optimizedmodel.params['smoothing_level'],DHES_optimizedmodel.params['smoothing_trend'],DHES_optimizedmodel.params['damping_trend']))
print("Optimal parameters best found by grid search method:: Alpha={:.1f}, Beta={:.1f},Gamma={:.2f}".format(alpha[DHESbest[0][0]],beta[DHESbest[1][0]],DHES_bestmodel.params['damping_trend']))

In [ ]:
plt.figure(figsize=(12, 8))
plt.plot(y_t[490:], color="black")
plt.plot(DHES_bestmodel.forecast(10), color="blue",label = "found by grid search method:alpha={:.1f}, beta={:.1f},Gamma={:.1f}".format(alpha[DHESbest[0][0]],beta[DHESbest[1][0]],DHES_bestmodel.params['damping_trend']))
plt.plot(DHES_optimizedmodel.forecast(10), color="red",label = "found by python function : alpha={:.2f}, beta={:.2f},Gamma={:.2f}".format(DHES_optimizedmodel.params['smoothing_level'],DHES_optimizedmodel.params['smoothing_trend'],DHES_optimizedmodel.params['damping_trend']))
plt.legend()
plt.show()

**Comparing the forecasts of all methods using the models of grtid search**

In [ ]:
plt.plot(y_t[490:], color="black")
plt.plot(SES_bestmodel.forecast(10),label = "alpha={:.1f}".format(SES_bestmodel.params["smoothing_level"]))
plt.plot(HES_bestmodel.forecast(10),label = "found by grid search method:alpha={:.1f}, beta={:.1f}".format(HES_bestmodel.params["smoothing_level"],HES_bestmodel.params["smoothing_trend"]))
plt.plot(DHES_bestmodel.forecast(10), color="blue",label = "found by grid search method:alpha={:.1f}, beta={:.1f},Gamma={:.1f}".format(alpha[DHESbest[0][0]],beta[DHESbest[1][0]],DHES_bestmodel.params['damping_trend']))